In [15]:
import csv
import codecs
import subprocess
import os
import re
import glob

# get the p-value from the original diwan sheet to be attached to the 236 names
pval_by_name = dict()
#with codecs.open("diwan_missing_archaea.csv", "r",encoding='utf-8', errors='ignore') as f:
with codecs.open("diwan_missing_bacteria.csv", "r",encoding='utf-8', errors='ignore') as f:
    for row in f:
        if (row.startswith("Species")):
            continue
        data = row.strip().split(",")
        name = data[0].strip()
        p_val = data[14]
        #print (name + " " + str(p_val))
        species_dict_by_name[name] = p_val

#with codecs.open("Diwan_MissingArchaea_FilesInfo.csv", "r",encoding='utf-8', errors='ignore') as f:
 #   with open('Diwan_MissingArchaea_FilesInfo_withPvalshuffle.csv', 'w') as fout:
with codecs.open("Diwan_MissingBacteria_FilesInfo.csv", "r",encoding='utf-8', errors='ignore') as f:
    with open('Diwan_MissingBacteria_FilesInfo_withPvalshuffle.csv', 'w') as fout:
        csvWriter = csv.writer(fout, delimiter=',')
        csvWriter.writerow(["name","full_file_prefix", "16S_end", "Diwan_SD", "pvalshuff (aff5 to highest)"])
        for row in f:
            if (row.startswith("name")):
                continue
            data = row.strip().split(",")
            name = data[0].strip()
            data.append(species_dict_by_name[name])
            #print(data)
            csvWriter.writerow(data)

In [13]:
import csv
import codecs
import subprocess
import os
import re
import glob

# get the p-value from the original diwan sheet to be attached to the 236 names
pval_by_name = dict()
with codecs.open("DiwanSupplementOriginal.csv", "r",encoding='utf-8', errors='ignore') as f:
    for row in f:
        if (row.startswith("Species")):
            continue
        data = row.strip().split(",")
        name = data[0].strip()
        p_val = data[14]
        #print (name + " " + str(p_val))
        species_dict_by_name[name] = p_val
        
with codecs.open("DiwanDataFilesInfo.csv", "r",encoding='utf-8', errors='ignore') as f:
    with open('DiwanDataFilesInfo_withPvalshuffle.csv', 'w') as fout:
        csvWriter = csv.writer(fout, delimiter=',')
        csvWriter.writerow(["name","full_file_prefix", "16S_end", "Diwan_SD", "pvalshuff (aff5 to highest)"])
        for row in f:
            if (row.startswith("name")):
                continue
            data = row.strip().split(",")
            name = data[0].strip()
            data.append(species_dict_by_name[name])
            #print(data)
            csvWriter.writerow(data)

In [17]:
import csv
import codecs
import subprocess
import os
import re
import glob

# process the data file to run the india paper stuff - just run the processing on the 48 files not found in bacteria
# on the previous run

species_names = []
species_dict_by_name = dict()
with codecs.open("diwan_missing_archaea.csv", "r",encoding='utf-8', errors='ignore') as f:
    for row in f:
        if (row.startswith("Species")):
            continue
        data = row.strip().split(",")
        name = data[0].strip()
        diwan_anti_sd = data[8][1:-1].upper()
        diwan_anti_sd = diwan_anti_sd.replace('U', 'T')
        revcompl = lambda x: ''.join([{'A':'T','C':'G','G':'C','T':'A'}[B] for B in x][::-1])
        diwan_sd = revcompl(diwan_anti_sd)
        #if (diwan_sd != "AGGAGG"):
         #   print(name, " ", diwan_sd)
        species_names.append(name)
        species_dict_by_name[name] = dict()
        species_dict_by_name[name]["diwan_sd"] = diwan_sd
        species_dict_by_name[name]["found_data_files"] = False
        species_dict_by_name[name]["assebly_file_prefix"] = ""
        species_dict_by_name[name]["16S_end"] = ""
        species_dict_by_name[name]["original_row"] = row
        
        
# read the large excel file, and find the correct genome
with codecs.open("/scratch4/moamin/ncbi_archaea_db/archaea_helix45_16s18_organism_information.csv", "r",encoding='utf-8', errors='ignore') as f:
    for row in f:
        data = row.strip().split(",")
        name = data[3].strip()
        if ((name in species_dict_by_name) and ("Complete Genome" in data[1].strip())):
            species_dict_by_name[name]["16S_end"] = data[23]
            prefix = data[0]
            species_dict_by_name[name]["assebly_file_prefix"] = prefix
            species_dict_by_name[name]["found_data_files"] = True

count = 0
with open('Diwan_MissingArchaea_FilesInfo.csv', 'w') as f:
    csvWriter = csv.writer(f, delimiter=',')
    csvWriter.writerow(["name","full_file_prefix", "16S_end", "Diwan_SD"])
    # count missing names, and get the full file path, output everything to a csv file
    for name in species_names:
        if (species_dict_by_name[name]["found_data_files"] == False):
            #print("Not Found for " + name)
            print(species_dict_by_name[name]["original_row"])
            count += 1
        else:
            prefix = species_dict_by_name[name]["assebly_file_prefix"]
            #files = [f for f in os.listdir('/scratch4/moamin/ncbi_bacteria_db/GbBac_GFF/') if re.match(r'*.gz', f)]
            names= glob.glob('/scratch4/moamin/ncbi_archaea_db/GbArchaea_GFF/' + prefix + '*.gz')
            if (len(names) > 1):
                print("Error, more than 1 file with prefix:", first)
            else:
                full_prefix = os.path.split(names[0])[1]
                full_prefix = re.sub('_genomic\.gff\.gz$', '', full_prefix)
                #print(full_prefix)
                csvWriter.writerow([name, full_prefix, species_dict_by_name[name]["16S_end"], species_dict_by_name[name]["diwan_sd"]])
        
# getting 236 out of 285 species (found no matches for 48 species)
print("Not found matches for ", count, " species")

Not found matches for  0  species


In [20]:
import csv
import codecs
import subprocess
import os
import re
import glob

# process the data file to run the india paper stuff

species_names = []
species_dict_by_name = dict()
with codecs.open("diwan_missing_bacteria.csv", "r",encoding='utf-8', errors='ignore') as f:
#with codecs.open("DiwanSupplementOriginal.csv", "r",encoding='utf-8', errors='ignore') as f:
    for row in f:
        if (row.startswith("Species")):
            continue
        data = row.strip().split(",")
        name = data[0].strip()
        diwan_anti_sd = data[8][1:-1].upper()
        diwan_anti_sd = diwan_anti_sd.replace('U', 'T')
        revcompl = lambda x: ''.join([{'A':'T','C':'G','G':'C','T':'A'}[B] for B in x][::-1])
        diwan_sd = revcompl(diwan_anti_sd)
        #if (diwan_sd != "AGGAGG"):
         #   print(name, " ", diwan_sd)
        species_names.append(name)
        species_dict_by_name[name] = dict()
        species_dict_by_name[name]["diwan_sd"] = diwan_sd
        species_dict_by_name[name]["found_data_files"] = False
        species_dict_by_name[name]["assebly_file_prefix"] = ""
        species_dict_by_name[name]["16S_end"] = ""
        species_dict_by_name[name]["original_row"] = row
        
        
# read the large excel file, and find the correct genome
with codecs.open("/scratch4/moamin/ncbi_bacteria_db/general_helix45_16s18_organism_information.csv", "r",encoding='utf-8', errors='ignore') as f:
    for row in f:
        data = row.strip().split(",")
        name = data[3].strip()
        #if (name in species_dict_by_name):
        if ((name in species_dict_by_name) and ("Complete Genome" in data[1].strip())):
            species_dict_by_name[name]["16S_end"] = data[23]
            prefix = data[0]
            species_dict_by_name[name]["assebly_file_prefix"] = prefix
            species_dict_by_name[name]["found_data_files"] = True

count = 0
with open('Diwan_MissingBacteria_FilesInfo.csv', 'w') as f:
#with open('DiwanDataFilesInfo.csv', 'w') as f:
    csvWriter = csv.writer(f, delimiter=',')
    csvWriter.writerow(["name","full_file_prefix", "16S_end", "Diwan_SD"])
    # count missing names, and get the full file path, output everything to a csv file
    for name in species_names:
        if (species_dict_by_name[name]["found_data_files"] == False):
            #print("Not Found for " + name)
            print(species_dict_by_name[name]["original_row"])
            count += 1
        else:
            prefix = species_dict_by_name[name]["assebly_file_prefix"]
            #files = [f for f in os.listdir('/scratch4/moamin/ncbi_bacteria_db/GbBac_GFF/') if re.match(r'*.gz', f)]
            names= glob.glob('/scratch4/moamin/ncbi_bacteria_db/GbBac_GFF/' + prefix + '*.gz')
            if (len(names) > 1):
                print("Error, more than 1 file with prefix:", first)
            else:
                full_prefix = os.path.split(names[0])[1]
                full_prefix = re.sub('_genomic\.gff\.gz$', '', full_prefix)
                #print(full_prefix)
                csvWriter.writerow([name, full_prefix, species_dict_by_name[name]["16S_end"], species_dict_by_name[name]["diwan_sd"]])
        
# getting 236 out of 285 species (found no matches for 48 species)
print("Not found matches for ", count, " species")

Not found matches for  0  species


In [10]:
import csv
import codecs
import subprocess
import os
import re
import glob


# process the data file to run pipeline on Ruhul's 153 files
with open('Ruhul_Annotate_153_DataFilesInfo.csv', 'w') as f:
    csvWriter = csv.writer(f, delimiter=',')
    csvWriter.writerow(["name","full_file_prefix", "16S_end", "Diwan_SD"])
    # read the large excel file, and find the correct genome
    with codecs.open("ruhul_re-annotation_all_organism_information.csv", "r",encoding='utf-8', errors='ignore') as f:
        for row in f:
            data = row.strip().split(",")
            name = data[3].strip()
            prefix = data[0]
            end_of_16S = data[23]
            names= glob.glob('/scratch4/moamin/ncbi_bacteria_db/GbBac_GFF/' + prefix + '*.gz')
            if (len(names) > 1):
                print("Error, more than 1 file with prefix:", first)
            else:
                full_prefix = os.path.split(names[0])[1]
                full_prefix = re.sub('_genomic\.gff\.gz$', '', full_prefix)
                #print(full_prefix)
                csvWriter.writerow([name, full_prefix, end_of_16S, ""])       
             

In [23]:
import csv
import codecs
import subprocess
import os
import re
import glob


# process the data file to run pipeline on original files from the paper

#with open('Our_NOT_IN_DIWAN_SD_Bacteria_DataFilesInfo.csv', 'w') as f:
with open('Our_NOT_IN_DIWAN_SD_Archaea_DataFilesInfo.csv', 'w') as f:
    csvWriter = csv.writer(f, delimiter=',')
    csvWriter.writerow(["name","full_file_prefix", "16S_end", "Diwan_SD"])
    # read the large excel file, and find the correct genome
    #with codecs.open("our_from_paper_not_in_diwan_bacteria.csv", "r",encoding='utf-8', errors='ignore') as f:
    with codecs.open("our_from_paper_not_in_diwan_archaea.csv", "r",encoding='utf-8', errors='ignore') as f:
        for row in f:
            data = row.strip().split(",")
            name = data[3].strip()
            prefix = data[0]
            end_of_16S = data[23]
            #names= glob.glob('/scratch4/moamin/ncbi_bacteria_db/GbBac_GFF/' + prefix + '*.gz')
            names= glob.glob('/scratch4/moamin/ncbi_archaea_db/GbArchaea_GFF/' + prefix + '*.gz')
            if (len(names) > 1):
                print("Error, more than 1 file with prefix:", first)
            else:
                print(names)
                print(prefix)
                full_prefix = os.path.split(names[0])[1]
                full_prefix = re.sub('_genomic\.gff\.gz$', '', full_prefix)
                #print(full_prefix)
                csvWriter.writerow([name, full_prefix, end_of_16S, ""])       
             



['/scratch4/moamin/ncbi_archaea_db/GbArchaea_GFF/GCA_000191585.1_ASM19158v1_genomic.gff.gz']
GCA_000191585.1
['/scratch4/moamin/ncbi_archaea_db/GbArchaea_GFF/GCA_000306765.2_ASM30676v2_genomic.gff.gz']
GCA_000306765.2
['/scratch4/moamin/ncbi_archaea_db/GbArchaea_GFF/GCA_000017185.1_ASM1718v1_genomic.gff.gz']
GCA_000017185.1


In [6]:
import csv
import codecs
import subprocess
import os
import re
import glob


# process the 166 bad tompa entries to separate archaea from bacteria

#with open('166_bad_tompa_Bacteria_DataFilesInfo.csv', 'w') as f_bacteria:
 #   with open('166_bad_tompa_Archaea_DataFilesInfo.csv', 'w') as f_archaea:
with open('all_just_tompa_Bacteria_DataFilesInfo.csv', 'w') as f_bacteria:
    with open('all_just_tompa_Archaea_DataFilesInfo.csv', 'w') as f_archaea:
        csvWriter_bacteria = csv.writer(f_bacteria, delimiter=',')
        csvWriter_bacteria.writerow(["name","full_file_prefix", "16S_end", "Diwan_SD"])
        csvWriter_archaea = csv.writer(f_archaea, delimiter=',')
        csvWriter_archaea.writerow(["name","full_file_prefix", "16S_end", "Diwan_SD"])
        # read the large excel file, and find the correct genome
        #with codecs.open("our_from_paper_not_in_diwan_bacteria.csv", "r",encoding='utf-8', errors='ignore') as f:
        #with codecs.open("166_bad_tompa.csv", "r",encoding='utf-8', errors='ignore') as f:
        with codecs.open("all_just_tompa.csv", "r",encoding='utf-8', errors='ignore') as f:
            for row in f:
                data = row.strip().split(",")
                name = data[0]
                full_prefix = data[1]
                end_of_16S = data[2]
                names= glob.glob('/scratch4/moamin/ncbi_bacteria_db/GbBac_GFF/' + full_prefix + '*.gz')
                if (len(names) > 1):
                    print("Error, in Bacteria, more than 1 file with prefix: ", full_prefix)
                else:
                    if (len(names) > 0):
                        csvWriter_bacteria.writerow(data) #[name, full_prefix, end_of_16S, ""])       
                    else:
                        names= glob.glob('/scratch4/moamin/ncbi_archaea_db/GbArchaea_GFF/' + full_prefix + '*.gz')
                        if (len(names) > 1):
                            print("Error, in Archaea, more than 1 file with prefix: ", full_prefix)
                        else:
                            if (len(names) > 0):
                                csvWriter_archaea.writerow(data) #[name, full_prefix, end_of_16S, ""])
                            else:
                                print("Error, name missing from both bacteria and archaea: ", full_prefix)

Error, name missing from both bacteria and archaea:  full_file_prefix


In [7]:
# post-processing - is SD that is found in the tail?
import csv
import codecs
import subprocess
import os
import re
import glob

# process the data file to run pipeline on our stuff


#with open('DiwanDataFiles_WithAllStats_236_AGGAGG_and_Tompa_v4_rounded_annotated.csv', 'w') as f:
#with open('Ruhul_Annotate_153_DataFiles_WithAllStats_only_tompa_v4_153_rounded_annotated.csv', 'w') as f:
#with open('Diwan_Missing_48_allstats_rounded_annotated.csv', 'w') as f:
#with open('166_missing_DataFiles_WithAllStats_only_tompa_v4_rounded_annotated.csv', 'w') as f:
with open('all_just_tompa_WithAllStats_withTompa_v4_rounded_annotated.csv', 'w') as f:
    csvWriter = csv.writer(f, delimiter=',')
    # read the large excel file, and find the correct genome
    #with codecs.open("Diwan_Missing_48_allstats.csv", "r",encoding='utf-8', errors='ignore') as f:
    #with codecs.open("Ruhul_Annotate_153_DataFiles_WithAllStats_only_tompa_v4_153.csv", "r",encoding='utf-8', errors='ignore') as f: 
    #with codecs.open("DiwanDataFiles_WithAllStats_236_AGGAGG_and_Tompa_v4_rounded.csv", "r",encoding='utf-8', errors='ignore') as f:
    #with codecs.open("166_missing_allstats.csv", "r",encoding='utf-8', errors='ignore') as f:
    with codecs.open("all_just_tompa_WithAllStats_withTompa_v4.csv", "r",encoding='utf-8', errors='ignore') as f:
        for row in f:
            data = row.strip().split(",")
            if (data[0] == "name"):
                csvWriter.writerow(data + ["found in 16S"])
                continue
            seq = data[3]
            complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
            reverse_complement = "".join(complement.get(base, base) for base in reversed(seq))
            #print(reverse_complement)
            if (("-" not in seq) and ("-" not in data[2]) and (reverse_complement in data[2])):
                csvWriter.writerow(data + ["yes"])
            else:
                csvWriter.writerow(data + ["no"])
   

In [24]:
# post-processing;  get the z-score from profile-input (later move to SD processing...)
import csv
import codecs
import subprocess
import os
import re
import glob


#with open('all_just_tompa_just_z_scores.csv', 'w') as f:
with open('NoSD_HighZ_withTopPattern.csv', 'w') as f:
    csvWriter = csv.writer(f, delimiter=',')
    #with codecs.open("all_just_tompa_WithAllStats_withTompa_v4_rounded_annotated.csv", "r",encoding='utf-8', errors='ignore') as f:
    with codecs.open("NoSD_HighZ.csv", "r",encoding='utf-8', errors='ignore') as f: 
        for row in f:
            data = row.strip().split(",")
            if (data[0] == "name"):
                #csvWriter.writerow(["name,full_file_prefix,zscore"])
                csvWriter.writerow(data + ["top pattern"])
            else:
                profile_input = data[1] + "_profile_input"
                count = 1
                zscore = -1
                zpattern = "-------"
                try:
                    with open(profile_input, "r+") as f:
                        for line in f:
                            numbers = line.strip().split()
                            score = numbers[-1]
                            pattern = numbers[0]
                            if (float(score) > float(zscore)):
                                zscore = score
                                zpattern = pattern
                            if (count == 20):
                                break
                            else:
                                count += 1
                except OSError as e:
                    zscore = -1
                    zpattern = "-------"
                #csvWriter.writerow(data[:2] + [zscore])
                csvWriter.writerow(data + [zpattern])
                print(zscore)
                print(zpattern)
                print("_____")
        

7.115
TACAATC
_____
-1
-------
_____
19.73
CTTCTTC
_____
7.831
CATTAAC
_____
8.775
AAAACCA
_____
8.624
CCGGCCG
_____
-1
-------
_____
6.318
CCGCATG
_____
12.38
GGTGCAG
_____
18.89
CGCGCGC
_____
11.59
GATCCCG
_____
14.52
CCCGCTG
_____
12.67
GCTCCTC
_____
14.27
CGCGCAG
_____
13.52
GCCGACG
_____
12.41
CTGACAC
_____
17.01
GGGCGTG
_____
-1

_____
9.245
CCCCGCC
_____
8.066
AGCGCTA
_____
14.04
GAACGAG
_____
10.77
TCCTTGG
_____
6.767
CGGAGCG
_____
14.79
CCGGTTC
_____
14.2
CCGTCGT
_____
12.06
CGGGGGC
_____
5.053
CGGTGCT
_____
22.41
ATCTTTG
_____
7.804
ACCACCA
_____
-1
-------
_____
-1
-------
_____
10.91
AAGGAGG
_____
-1
-------
_____
5.694
GTATAAT
_____
7.715
AGAAAAA
_____
8.412
ACAAAAA
_____
8.492
GGAGGGC
_____
6.01
GCTACAA
_____


In [88]:
# process the sheet for the FDR with benjamin-hochberg
# same package is 
# http://www.statsmodels.org/dev/generated/statsmodels.sandbox.stats.multicomp.fdrcorrection0.html#statsmodels.sandbox.stats.multicomp.fdrcorrection0
import mne.stats
import numpy as np
import csv

#alpha = 0.05
#pval = [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001,0.0001,0.0001,0.0001,0.0001, 0.8001, 0.8901,]
#reject_fdr, pval_fdr = mne.stats.fdr_correction(pval, alpha=alpha, method='indep')
#print(pval)
#print(reject_fdr)
#print(pval_fdr)

pvals_tompa = []
pvals_diwan = []
with codecs.open("Oct_3_Ruhul153_Plus_Diwan284_Plus_Original10_AllStats_only_tompa.csv", "r",encoding='utf-8', errors='ignore') as f: 
    for row in f:
        data = row.strip().split(",")
        if (data[0] != "name"):
            if (data[7] != '-1'):
                pvals_tompa.append(data[7])
            pvals_diwan.append(data[11])
print(len(pvals_tompa))
print(len(pvals_diwan))
pvals_tompa = np.array(pvals_tompa).astype(np.float)
pvals_diwan = np.array(pvals_diwan).astype(np.float)
reject_fdr_tompa, pval_fdr_tompa = mne.stats.fdr_correction(pvals_tompa, alpha=alpha, method='indep')
reject_fdr_diwan, pval_fdr_diwan = mne.stats.fdr_correction(pvals_diwan, alpha=alpha, method='indep')
print(reject_fdr_tompa)
with open('Oct_3_Ruhul153_Plus_Diwan284_Plus_Original10_AllStats_only_tompa_withFDR.csv', 'w') as f:
    csvWriter = csv.writer(f, delimiter=',')
    with codecs.open("Oct_3_Ruhul153_Plus_Diwan284_Plus_Original10_AllStats_only_tompa.csv", "r",encoding='utf-8', errors='ignore') as f: 
        idx_tompa = 0
        idx_diwan = 0
        for row in f:
            data = row.strip().split(",")
            if (data[0] == "name"):
                csvWriter.writerow(data + ["Combined P-value Accepted", "Combined P-value FDR Corrected", "Diwan P-value Accepted", "Diwan P-value FDR Corrected"])
            else:
                append_tompa = []
                if (data[7] != '-1'):
                    append_tompa = [reject_fdr_tompa[idx_tompa], pval_fdr_tompa[idx_tompa]]
                    idx_tompa += 1
                else:
                    append_tompa = ["N/A", "-1"]
                append_diwan = [reject_fdr_diwan[idx_diwan], pval_fdr_diwan[idx_diwan]]
                csvWriter.writerow(data + append_tompa + append_diwan)
                idx_diwan += 1

413
447
[ True False  True False False  True False False False False False False
 False False False False False False  True False False False False False
 False False False False  True False  True  True False False False False
 False  True False False False False False  True False False False False
 False False False False  True False False False False False False False
 False False False False False False False  True  True False False False
 False False  True False False False False  True  True False  True  True
 False False False False False False False False False False False False
 False False False False False  True False False False False  True False
  True  True False  True  True  True  True False False False  True False
 False  True False False False  True  True False False False False  True
 False  True False  True False False False False False False False False
 False False False False False False False False False False False False
 False False False  True  True False False 

In [4]:
%%time

# This is only for codon pair p value

# process the sheet for the FDR with benjamin-hochberg
# same package is 
# http://www.statsmodels.org/dev/generated/statsmodels.sandbox.stats.multicomp.fdrcorrection0.html#statsmodels.sandbox.stats.multicomp.fdrcorrection0
import codecs
import mne.stats
import numpy as np
import csv

alpha = 0.05
#pval = [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001,0.0001,0.0001,0.0001,0.0001, 0.8001, 0.8901,]
#reject_fdr, pval_fdr = mne.stats.fdr_correction(pval, alpha=alpha, method='indep')
#print(pval)
#print(reject_fdr)
#print(pval_fdr)

pvals_tompa = []
with codecs.open("Oct_3_Ruhul153_Plus_Diwan284_Plus_Original10_AllStats_only_original_withFDR.csv", "r",encoding='utf-8', errors='ignore') as f: 
    for row in f:
        data = row.strip().split(",")
        if (data[0] != "name"):
            if (data[7] != '-1'):
                pvals_tompa.append(data[5])
print(len(pvals_tompa))
pvals_tompa = np.array(pvals_tompa).astype(np.float)
reject_fdr_tompa, pval_fdr_tompa = mne.stats.fdr_correction(pvals_tompa, alpha=alpha, method='indep')
print(reject_fdr_tompa)
with open('Oct_3_Ruhul153_Plus_Diwan284_Plus_Original10_AllStats_only_original_with_Updated_FDR.csv', 'w') as f:
    csvWriter = csv.writer(f, delimiter=',')
    with codecs.open("Oct_3_Ruhul153_Plus_Diwan284_Plus_Original10_AllStats_only_original_withFDR.csv", "r",encoding='utf-8', errors='ignore') as f: 
        idx_tompa = 0
        for row in f:
            data = row.strip().split(",")
            if (data[0] == "name"):
                csvWriter.writerow(data + ["Codon Pair P-value Accepted", "Codon Pair P-value FDR Corrected"])
            else:
                append_tompa = []
                if (data[7] != '-1'):
                    append_tompa = [reject_fdr_tompa[idx_tompa], pval_fdr_tompa[idx_tompa]]
                    idx_tompa += 1
                else:
                    append_tompa = ["N/A", "-1"]
                csvWriter.writerow(data + append_tompa)

443
[False False False False False  True False False False False False False
 False False False False False  True False False False False False False
 False False False False False False False False  True False False False
 False False  True False False False  True False False  True  True  True
  True  True False  True False False False False False False False False
 False False False False False False False False False False False False
 False False  True  True  True False  True  True  True  True False False
 False False False False False False False  True False False False  True
 False  True False  True False False False False False False False False
 False False False False False False False False  True False False False
  True  True False False False False False  True  True  True False False
 False False  True  True False False  True False False False False False
 False False False False False False  True False False False False False
 False  True False False False False False Fals

In [84]:
# fix the 16S annotations in the final spreadsheet

bacteria_name_to_16S_dict = dict()
# for the bacteria; TODO:  add for archaea
with codecs.open("refined_annotation_helix45_16s18_per_organism.csv", "r",encoding='utf-8', errors='ignore') as f:
    for row in f:
        data = row.strip().split(",")
        name = data[3].strip()
        if (name in bacteria_name_to_16S_dict):
            if (data[23] != bacteria_name_to_16S_dict[name]):
                if (bacteria_name_to_16S_dict[name] == "-------------"):
                    bacteria_name_to_16S_dict[name] = data[23]
        else:
            bacteria_name_to_16S_dict[name] = data[23]
            #print(name + " " + data[24])

archaea_name_to_16S_dict = dict()
# for the bacteria; TODO:  add for archaea
with codecs.open("archaea_refined_annotation_per_organism.csv", "r",encoding='utf-8', errors='ignore') as f:
    for row in f:
        data = row.strip().split(",")
        name = data[3].strip()
        if (name in archaea_name_to_16S_dict):
            if (data[23] != archaea_name_to_16S_dict[name]):
                if (archaea_name_to_16S_dict[name] == "-------------"):
                    archaea_name_to_16S_dict[name] = data[23]
        else:
            archaea_name_to_16S_dict[name] = data[23]
            #print(name + " " + data[24])
            

found_bacteria = 0
found_archaea = 0
different_16S_bacteria = 0
different_16S_archaea = 0
with open('Oct_10_Ruhul153_Plus_Diwan284_Plus_Original10_AllStats.csv', 'w') as f:
    csvWriter = csv.writer(f, delimiter=',')
    with codecs.open("Oct_1_Ruhul153_Plus_Diwan284_Plus_Original10_AllStats.csv", "r",encoding='utf-8', errors='ignore') as f:
        for row in f:
            data = row.strip().split(",")
            if (data[0] == "name"):
                csvWriter.writerow(data)
            else:
                name = data[0].strip()
                new_16S = ""
                if (name in bacteria_name_to_16S_dict):
                    found_bacteria += 1
                    if (data[2] != bacteria_name_to_16S_dict[name]):
                        different_16S_bacteria += 1
                        print(name + ": " + data[2] + " " + bacteria_name_to_16S_dict[name])
                        new_16S = bacteria_name_to_16S_dict[name]
                if (name in archaea_name_to_16S_dict):
                    found_archaea += 1
                    if (data[2] != archaea_name_to_16S_dict[name]):
                        different_16S_archaea += 1
                        print(name + ": " + data[2] + " " + archaea_name_to_16S_dict[name])
                        new_16S = archaea_name_to_16S_dict[name]
                if (new_16S != ""):
                    data[2] = new_16S
                    seq = data[3]
                    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
                    reverse_complement = "".join(complement.get(base, base) for base in reversed(seq))
                    #print(reverse_complement)
                    if (("-" not in seq) and ("-" not in data[2]) and (reverse_complement in data[2])):
                        data[9] = "yes"
                    else:
                        data[9] = "no"
                    csvWriter.writerow(data)
print("Found " + str(found_bacteria) + " bacteria by name")
print("Will update " + str(different_16S_bacteria) + " bacteria")
print("Found " + str(found_archaea) + " archaea by name")
print("Will update " + str(different_16S_archaea) + " archaea")

Burkholderia thailandensis E264: ------------- GATCACCTCCTTT
Pseudomonas aeruginosa PAO1: ------------- GATCACCTCCTTA
Rhodobacter sphaeroides 2.4.1: ------------- GATCACCTCCTTT
Vibrio cholerae O1 biovar El Tor str. N16961: ------------- GATCACCTCCTTA
Burkholderia thailandensis E264: ------------- GATCACCTCCTTT
Pseudomonas aeruginosa PAO1: ------------- GATCACCTCCTTA
Rhodobacter sphaeroides 2.4.1: ------------- GATCACCTCCTTT
Vibrio cholerae O1 biovar El Tor str. N16961: ------------- GATCACCTCCTTA
Yersinia pestis CO92: ------------- GATCACCTCCTTA
Yersinia pestis CO92: ------------- GATCACCTCCTTA
Corynebacterium glutamicum ATCC 13032: ------------- GATCACCTCCTTT
Helicobacter pylori J99: ------------- GATCACCTCCTTT
Corynebacterium glutamicum ATCC 13032: ------------- GATCACCTCCTTT
Helicobacter pylori J99: ------------- GATCACCTCCTTT
Bacillus subtilis subsp. subtilis str. 168: ------------- GATCACCTCCTTT
Bacillus subtilis subsp. subtilis str. 168: ------------- GATCACCTCCTTT
Found 830 bact